In [8]:
import sys
import os
pathap = os.path.join('GENE_ML','gene_ml','static_sparse_grid_approximations')
print('PATHAP',pathap)
sys.path.append(pathap)

#allows any changes on external files to be loaded on code run.
%load_ext autoreload
%autoreload 2


PATHAP GENE_ML/gene_ml/static_sparse_grid_approximations


In [6]:
import numpy as np


### parameter bounds
# temp ratio
left_temp  	= 0.2
right_temp 	= 1.3

# grad n
left_omn  	= -0.4
right_omn   = 2.4

# grad T_i
left_omt_ions  	= 2.
right_omt_ions 	= 4.

# grad T_e
left_omt_elecs  	= 1.
right_omt_elecs  	= 3.

left_stoch_boundary 	= [left_temp, left_omn, left_omt_ions, left_omt_elecs]
right_stoch_boundary 	= [right_temp, right_omn, right_omt_ions, right_omt_elecs]


### sparse grid setup
dim 			= 4 # no of input parameters of interest 
level_to_nodes 	= 2 # this parameter determines the growth of the Leja points. A value of 1 indicates that we add only one extra point per level
level 			= 3 # with this setup, the maxium degree of the 1D monomials will be level - 1
######


### setup for the standard uniform distribution
weights 		= [lambda x: 1. for d in range(dim)]
left_bounds    	= np.zeros(dim)
right_bounds   	= np.ones(dim)
######


### mapping from [0, 1] -> [a, b]
mapping_to_a_b = lambda a, b, x: a + (b - a)*x
######

### other config steps

In [24]:
## target-function
f_ref = lambda x: x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + 2*x[2] * x[3] + 3

parameters = ['temp','grad_n','grad_Ti','grad_Te']
bounds = list(zip(left_stoch_boundary, right_stoch_boundary))

from GENE_ML.gene_ml.samplers.static_sparse_grid import StaticSparseGrid
sampler_ssg = StaticSparseGrid(parameters=parameters,bounds=bounds, level=level)

from GENE_ML.gene_ml.dataset.base import DataSet
data_set = DataSet()
data_set.x = np.array([np.array(l) for l in list(sampler_ssg.samples.values())])
data_set.growthrates = f_ref(data_set.x)

data_set.growthrates

from GENE_ML.gene_ml.models.SSG_POLY import SSG_POLY
name='ssg_poly'
poly = SSG_POLY(ssg_sampler=sampler_ssg, name='ssg_test')
poly.train(y=data_set.growthrates)

from GENE_ML.gene_ml.samplers.uniform import Uniform
sampler_unif = Uniform(bounds, 10, parameters)

true = f_ref(sampler_unif.samples_array.T)

predicted = poly.predict(sampler_unif.samples_array)

print(true)
print(predicted)

POINTS SHAPE (15, 4)
 no points for dim = 4 and level = 3 is n = 15
PREDICTION WALL TIME FOR 10 POINTS, dd-hh:mm:ss | 00-00:00:0
[40.8669361  42.35858348 24.62476631 29.64587101 26.96841593 36.35983109
 29.07008274 38.06044794 32.54417308 20.45178841]
[40.8669361  42.35858348 24.62476631 29.64587101 26.96841593 36.35983109
 29.07008274 38.06044794 32.54417308 20.45178841]
